# Comparison Using Own Dataset

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def predict(image_representation_list, true_label, test_representation):
    minimum_label = None
    minimum_distance = np.inf
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

## Test New Class using Own Dataset

In [3]:
import os

In [4]:
img_path_list = os.listdir('../face_data')
img_path_list = [os.path.join('../face_data', img_path) for img_path in img_path_list]

In [5]:
target = []
for img_path in img_path_list:
    if ('/1_' in img_path):
        target.append(1)
    elif ('/2_' in img_path):
        target.append(2)
    elif ('/7_' in img_path):
        target.append(3)
    elif ('/12_' in img_path):
        target.append(3)
    elif ('/17_' in img_path):
        target.append(4)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train_img_path_list, X_test_img_path_list, y_train, y_test = train_test_split(
    img_path_list, target, test_size=0.2, random_state=0
)

In [8]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

### Facenet

In [9]:
feature_extractor = FaceFeatureExtractor(FACENET_MODEL_PATH, extractor_name='facenet')

Model directory: model_data/facenet/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from model_data/facenet/20180402-114759/model-20180402-114759.ckpt-275


In [10]:
%%time
X_train_features = feature_extractor.extract_batch(X_train_img_path_list)
X_test_features = feature_extractor.extract_batch(X_test_img_path_list) 

CPU times: user 1min 24s, sys: 1min 9s, total: 2min 33s
Wall time: 24.9 s


In [11]:
# Must be closed to reset the graph dataflow
feature_extractor.extractor.close_session()

In [12]:
predict(X_train_features, y_train, X_test_features[0])

4

In [13]:
predicted_list = []
for test_feature in X_test_features:
    predicted_list.append(predict(X_train_features, y_train, test_feature))

In [14]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

1.0
1.0


### VGGFace

In [15]:
feature_extractor = FaceFeatureExtractor(VGGFACE_MODEL_PATH, extractor_name='vgg_face')

In [16]:
%%time
X_train_features = feature_extractor.extract_batch(X_train_img_path_list)
X_test_features = feature_extractor.extract_batch(X_test_img_path_list) 

CPU times: user 14min 14s, sys: 15.9 s, total: 14min 30s
Wall time: 1min 59s


In [17]:
predict(X_train_features, y_train, X_test_features[0])

4

In [18]:
predicted_list = []
for test_feature in X_test_features:
    predicted_list.append(predict(X_train_features, y_train, test_feature))

In [19]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.982995951417004


### VGGFace VGG16

In [20]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_vgg16')

In [21]:
%%time
X_train_features = feature_extractor.extract_batch(X_train_img_path_list)
X_test_features = feature_extractor.extract_batch(X_test_img_path_list) 

CPU times: user 4min 46s, sys: 6.71 s, total: 4min 53s
Wall time: 37.8 s


In [22]:
predict(X_train_features, y_train, X_test_features[0])

4

In [23]:
predicted_list = []
for test_feature in X_test_features:
    predicted_list.append(predict(X_train_features, y_train, test_feature))

In [24]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9821776663881926


### VGGFace Resnet50

In [25]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_resnet50')

In [26]:
%%time
X_train_features = feature_extractor.extract_batch(X_train_img_path_list)
X_test_features = feature_extractor.extract_batch(X_test_img_path_list) 

CPU times: user 8min 15s, sys: 1min 24s, total: 9min 40s
Wall time: 1min 20s


In [27]:
K.image_data_format()

'channels_last'

In [28]:
predict(X_train_features, y_train, X_test_features[0])

4

In [29]:
predicted_list = []
for test_feature in X_test_features:
    predicted_list.append(predict(X_train_features, y_train, test_feature))

In [30]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9820793814601864


### LBPH

In [31]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='lbph')

In [32]:
%%time
X_train_features = feature_extractor.extract_batch(X_train_img_path_list)
X_test_features = feature_extractor.extract_batch(X_test_img_path_list) 

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/facerec/feature.py:266: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  H = np.histogram(C, bins=2**self.lbp_operator.neighbors, range=(0, 2**self.lbp_operator.neighbors), normed=True)[0]


CPU times: user 2.87 s, sys: 8.07 ms, total: 2.88 s
Wall time: 2.95 s


In [33]:
predict(X_train_features, y_train, X_test_features[0])

4

In [34]:
predicted_list = []
for test_feature in X_test_features:
    predicted_list.append(predict(X_train_features, y_train, test_feature))

In [35]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9820793814601864
